In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('test.parquet')
df

,ORGAO_LICITANTE,NUM_LICITACAO,OBJ_LICITACAO,MODALIDADE_LICITACAO,PROCESSO,DATA_ABERTURA,TIPO_OBJ,VALOR_ESTIMADO,CODIGO_SISTEMA_COMPRAS,SISTEMA_COMPRAS,treated_text,IOB
0,None,249/00,AQUISIÇÃO DE MATERIAL DE CONSUMO ; MATERIAL PA...,CONVITE,None,24/07/00,AQUISIÇÃO,None,None,http://www fazenda df gov.br,CONVITE Nº 249/00 Objeto: AQUISIÇÃO DE MATERIA...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
1,None,1.437/2008,Contratação de empresa para prestação de servi...,PREGÃO ELETRÔNICO,053.001.258/2008,16 de janeiro de 2009,prestação de serviços,None,None,www.compras.df.gov.br,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 1.437/...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
2,None,5/2003,"Aquisição de material de consumo ( limpeza, co...",CONCORRENCIA,None,10.03.2003,None,None,None,www.fazenda.df.gov.br,AVISOS DE LICITAÇÃO CONCORRENCIA N.º 5/2003-Su...,O O O B-MODALIDADE_LICITACAO O O B-NUM_LICITAC...
3,None,50/2015,Aquisição de material consumo Termoplástico Te...,PREGÃO ELETRÔNICO,113.013360/2015,27 de janeiro de 2016,Aquisição,None,None,None,AVISO DE LICITAÇÃ NOVA DATA PREGÃO ELETRÔNICO ...,O O O O O B-MODALIDADE_LICITACAO I-MODALIDADE_...
4,None,964/2008,Contratação de firma especializada em confecçã...,PREGÃO ELETRÔNICO,220.000.570/2008,11 de setembro de 2008,Contratação,None,None,www.compras.df.gov.br,PREGÃO ELETRÔNICO Nº 964/2008. Objeto: Contrat...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
...,...,...,...,...,...,...,...,...,...,...,...,...
66,None,224/2014,"aquisição de materiais de copa, cozinha, limpe...",PREGÃO ELETRÔNICO,414.000.216/2014,None,aquisição,"116.236,56",None,http://www.compras.df.gov.br,AVISO DE LICITAÇÃO NOVA DATA PREGÃO ELETRÔNICO...,O O O O O B-MODALIDADE_LICITACAO I-MODALIDADE_...
67,ACOMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO...,144/2012,Registro de Preços para fornecimento de reagen...,PREGÃO ELETRÔNICO,092.006540/2012,16 de outubro de 2012,Registro de Preços,"204.314,58",None,www.licitacoes-e.com.br,PREGÃO ELETRÔNICO Nº 144/2012 ACOMPANHIA DE SA...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
68,COMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO ...,05/2007,aquisição de peças/acessórios rios para equipa...,TOMADA DE PREÇOS,00092.000.301/2007,09 de maio de 2007,aquisição,None,None,None,AVISO DE LICITAÇÕES TOMADA DE PREÇOS Nº 05/200...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
69,Centrais de Abastecimento do Distrito Federal S.A,05/2018,contratação de - empresa para a prestação de s...,PREGÃO ELETRÔNICO,071.000040/2018,None,contratação,"75.000,00",926245,www.comprasnet.gov.br,AVISO DE LICITAÇÃO EXCLUSIVA PARA ME E EPP PRE...,O O O O O O O O B-MODALIDADE_LICITACAO I-MODAL...


In [3]:
df.isnull().sum()

ORGAO_LICITANTE           59
NUM_LICITACAO              0
OBJ_LICITACAO              1
MODALIDADE_LICITACAO       0
PROCESSO                  19
DATA_ABERTURA              6
TIPO_OBJ                   2
VALOR_ESTIMADO            51
CODIGO_SISTEMA_COMPRAS    59
SISTEMA_COMPRAS           23
treated_text               0
IOB                        0
dtype: int64

In [4]:
ORGAO_LICITANTE = []
VALOR_ESTIMADO = []


for i in range(len(df)):
    if df['ORGAO_LICITANTE'][i] != None:
        ORGAO_LICITANTE.append(i)
    if df['VALOR_ESTIMADO'][i] != None:
        VALOR_ESTIMADO.append(i)

In [5]:
len(ORGAO_LICITANTE), len(VALOR_ESTIMADO)

(12, 20)

In [6]:
ORGAO_LICITANTE_1 = ORGAO_LICITANTE[:(len(ORGAO_LICITANTE)*50)//100]
ORGAO_LICITANTE_2 = ORGAO_LICITANTE[(len(ORGAO_LICITANTE)*50)//100:]

VALOR_ESTIMADO_set = set(VALOR_ESTIMADO).difference(set(ORGAO_LICITANTE_2))
VALOR_ESTIMADO_1 = list(VALOR_ESTIMADO_set)[:(len(VALOR_ESTIMADO_set)*50)//100]
VALOR_ESTIMADO_2 = set(list(VALOR_ESTIMADO_set)[(len(VALOR_ESTIMADO_set)*50)//100:]).difference(set(ORGAO_LICITANTE_1))


In [7]:
len(ORGAO_LICITANTE_1), len(ORGAO_LICITANTE_2), len(VALOR_ESTIMADO_1), len(VALOR_ESTIMADO_2)

(6, 6, 9, 6)

In [8]:
lic = []

lic.extend(ORGAO_LICITANTE_1)
lic.extend(VALOR_ESTIMADO_1)

lic = sorted(list(set(lic)))

not_lic = []

not_lic.extend(ORGAO_LICITANTE_2)
not_lic.extend(VALOR_ESTIMADO_2)

not_lic = sorted(list(set(not_lic)))

In [9]:
len(lic), len(not_lic)

(15, 12)

In [10]:
for i in lic:
    for j in not_lic:
        if i == j:
            print(i)

In [11]:
df_lic = df.iloc[lic]
df_not_lic = df.iloc[not_lic]

In [12]:
len(df_lic), len(df_not_lic)

(15, 12)

In [13]:
df_real = df.drop(df_lic.index)
df_real = df_real.drop(df_not_lic.index)

In [14]:
len(df_real)

44

In [15]:
testa = pd.concat([df_lic, df_real[:int(len(df)*0.5) - len(df_lic)]])
testb = pd.concat([df_real[int(len(df)*0.5) - len(df_lic):], df_not_lic])

In [16]:
len(testa), len(testb), len(df)

(35, 36, 71)

In [17]:
i = 0
for col in list(df.columns):
    print(f'{col}: {((len(testa) - list(testa.isnull().sum())[i]) * 100) / (len(df) - list(df.isnull().sum())[i])}%')
    i += 1

ORGAO_LICITANTE: 50.0%
NUM_LICITACAO: 49.29577464788732%
OBJ_LICITACAO: 50.0%
MODALIDADE_LICITACAO: 49.29577464788732%
PROCESSO: 50.0%
DATA_ABERTURA: 50.76923076923077%
TIPO_OBJ: 49.27536231884058%
VALOR_ESTIMADO: 60.0%
CODIGO_SISTEMA_COMPRAS: 50.0%
SISTEMA_COMPRAS: 50.0%
treated_text: 49.29577464788732%
IOB: 49.29577464788732%


In [18]:
i = 0
for col in list(df.columns):
    print(f'{col}: {((len(testb) - list(testb.isnull().sum())[i]) * 100) / (len(df) - list(df.isnull().sum())[i])}%')
    i += 1

ORGAO_LICITANTE: 50.0%
NUM_LICITACAO: 50.70422535211268%
OBJ_LICITACAO: 50.0%
MODALIDADE_LICITACAO: 50.70422535211268%
PROCESSO: 50.0%
DATA_ABERTURA: 49.23076923076923%
TIPO_OBJ: 50.72463768115942%
VALOR_ESTIMADO: 40.0%
CODIGO_SISTEMA_COMPRAS: 50.0%
SISTEMA_COMPRAS: 50.0%
treated_text: 50.70422535211268%
IOB: 50.70422535211268%


In [19]:
testa = testa.reset_index(drop=True)
testb = testb.reset_index(drop=True)

In [20]:
testa.to_parquet('testa.parquet')
testb.to_parquet('testb.parquet')

In [21]:
testa = pd.read_parquet('testa.parquet')
testb = pd.read_parquet('testb.parquet')

len(testa), len(testb)

(35, 36)